In [ ]:
import numpy as np
import cv2
import os
import pathlib

from typing import List

from ultralytics import YOLO
from ultralytics.engine.results import Boxes, Results

from objectherkenning_openbare_ruimte.data_minimisation import blurring_tools

PERSON = 0
LICENSE_PLATE = 1
CONTAINER = 2

target_classes = [CONTAINER]
sensitive_classes = [PERSON, LICENSE_PLATE]


def _get_annotion_string_from_boxes(boxes: Boxes) -> str:
    boxes = boxes.cpu()

    annotation_lines = []

    for box in boxes:
        cls = int(box.cls.squeeze())
        conf = float(box.conf.squeeze())
        tracking_id = int(box.id.squeeze()) if box.is_track else -1
        yolo_box_str = " ".join([f"{x:.6f}" for x in box.xywhn.squeeze()])
        annotation_lines.append(f"{cls} {yolo_box_str} {conf:.6f} {tracking_id}")
    return "\n".join(annotation_lines)


def _process_results(results: Results, orig_paths: List[str], destination_folder: str):
    for r, path in zip(results, orig_paths):
        result = r.cpu()
        boxes = result.boxes.numpy()

        target_idxs = np.where(np.in1d(boxes.cls, target_classes))[0]
        if len(target_idxs) == 0:  # Nothing to do!
            print(f"No containers found in {pathlib.Path(path).stem}")
            continue

        image = result.orig_img.copy()
        sensitive_idxs = np.where(np.in1d(boxes.cls, sensitive_classes))[0]

        # Blur sensitive data
        sensitive_bounding_boxes = boxes[sensitive_idxs].xyxy
        image = blurring_tools.blur_inside_boxes(image, sensitive_bounding_boxes)
        
        # Draw annotation boxes
        target_bounding_boxes = boxes[target_idxs].xyxy
        image = blurring_tools.draw_bounding_boxes(image, target_bounding_boxes)

        # Save image
        save_path = os.path.join(destination_folder, pathlib.Path(path).name)
        print(f"Writing image: {save_path}")
        cv2.imwrite(save_path, image)

        # Save annotation
        annotation_str = _get_annotion_string_from_boxes(boxes[target_idxs])
        base_name = pathlib.Path(path).stem
        annotation_path = os.path.join(destination_folder, f"{base_name}.txt")
        with open(annotation_path, 'w') as f:
            f.write(annotation_str)

In [ ]:

model = YOLO("../../datasets/oor/weights/yolov8s_coco_nc5_best.pt")

model_img_size = (1024, 1024)
output_img_size = (1280, 720)

input_image_folder = "../../datasets/oor/first-official-training-dataset-oor/images"
destination_folder = "../../datasets/oor/test_output"

pathlib.Path(destination_folder).mkdir(parents=True, exist_ok=True)

In [ ]:

img_path_list = sorted(list(pathlib.Path(input_image_folder).glob("*.jpg")))
img_path_list = img_path_list[10:20]

In [ ]:
for img_path in img_path_list:
    image = cv2.imread(img_path.as_posix(), cv2.IMREAD_COLOR)
    image = cv2.resize(image, output_img_size)
    result = model.predict(
        source=image,
        project="../../datasets/oor/first-official-training-dataset-oor/",
        imgsz=model_img_size,
        batch=16,
        conf=0.434,
        iou=0.7,
        save_txt=False,
        save_conf=True, 
    )
    _process_results(results=result, orig_paths=[img_path], destination_folder=destination_folder)